In [1]:
import rasterio
import numpy as np
import geopandas as gpd
import rioxarray # for the extension to load
import xarray as xr
import multiprocessing
import time
import gc

from rasterio.enums import Resampling

from sklearn_xarray import wrap
from sklearn.preprocessing import MinMaxScaler


# For larger rasters the two rasters need to be tiled (and should overlap!) 
# and the code below should be performed in sequence for each tile
# with 'just' 64 GB memory availabe it likely that it can be paralized in two sessions at maxmimum

In [4]:

# - Read rasters into memory -

# Use function 'reproject_match'
# Documentation: https://corteva.github.io/rioxarray/stable/examples/reproject_match.html
# Documentation: https://corteva.github.io/rioxarray/stable/rioxarray.html?highlight=write_crs#rioxarray.raster_array.RasterArray.reproject_match

# Load in xarray datasets
xds = rioxarray.open_rasterio(rasPathIn, mask_and_scale=True, cache=False, engine='rasterio') # target
xds_match = rioxarray.open_rasterio(rasPathTarget, mask_and_scale=True, cache=False, engine='rasterio') # Input


# Select only relevant bands from 10-band raster (RedEdge-MX Dual camera)
xds = xds[[1,3,5,7,9]]

In [6]:

# - Resample and match 5 multispectral bands to 1 cm RGB ortho

matched_bands = []
for b in range(len(xds)):
    t = time.time()
    xds_repr_match = xds[-1].rio.reproject_match(xds_match[0], resampling=Resampling.nearest)
    
    # From: https://corteva.github.io/rioxarray/stable/examples/reproject_match.html?highlight=stack
    # It is recommended to use assign_coords to make the coordinates the exact same due to 
    # tiny differences in the coordinate values due to floating precision 
    xds_repr_match = xds_repr_match.assign_coords({
        "x": xds_match.x,
        "y": xds_match.y,
    })
    
    matched_bands.append(xds_repr_match) # sampling on last band
    
    del(xds_repr_match) # Remove resampled band from memory
    xds[-1] = None # remove last band from memory
    gc.collect() # make memory available
    
    print('Band '+str(b),' done.')
    # Resample and match 5 multispectral bands to 1 cm RGB ortho
    print(time.time() - t)

Band 0  done.
218.85712885856628
Band 1  done.
27.853586196899414
Band 2  done.
30.623091220855713
Band 3  done.
30.883497953414917
Band 4  done.
33.36853575706482


In [7]:
# - Stack and save rasters

# Bands to be stacked
stacked_bands = np.stack([xds_match[0], xds_match[1], xds_match[2], 
             matched_bands[0], matched_bands[1], matched_bands[2],
             matched_bands[3], matched_bands[4]])



In [21]:
# Read metadata target raster
with rasterio.open(rasPathTarget) as src:
    profile = src.profile

# Edit metadata
profile['count'] = len(stacked_bands)
profile['nodata'] = np.nan
profile['dtype'] = np.float32    

In [30]:
rasPathOut =

# write stacked raster
with rasterio.open(rasPathOut, 'w', **profile) as src:
    src.write(stacked_bands)

In [1]:
# Finally NA values were 'filled' in ArcGIS Pro using raster calculator